In [ ]:
import calliope
import pandas as pd
import numpy as np


CONVERT_100_GW_to_GW = 100
CONVERT_10000km2_to_km2 = 10000
RENEWABLES = ["open_field_pv", "roof_mounted_pv", "wind", "awe"]


def get_tidy_data(model: calliope.Model, coefficient: str) -> pd.DataFrame:
    r"""
    Get data from a model and format it for plotting.
    """
    data = (
        model.get_formatted_array(coefficient, index_format="multiindex")
        .to_dataframe()
        .reset_index()
    )

    return data

def aggregate_locs(df):
    _df = df.copy()
    _df["locs"] = _df["locs"].replace({"_\\d+$": ""}, regex=True)
    _df = _df.groupby(["locs", "techs"]).sum().reset_index().set_index("locs")
    return _df


def filter_techs(df, filter_techs):
    if not isinstance(filter_techs, list):
        filter_techs = [filter_techs]
    return df.loc[df.techs.apply(lambda x: any([f in x for f in filter_techs]), 1),:]


def series_drop_nan_inf(series):
    return series.loc[~series.isna() & (~series.isin([np.inf, -np.inf]))]


def df_drop_nan_inf(df, axis, drop_if_any=True):
    if drop_if_any:
        condition = ~(df.isna() | df.isin([np.inf, -np.inf])).any(axis)
    else:
        condition = ~(df.isna() | df.isin([np.inf, -np.inf])).all(axis)
    if axis == 1:
        return df.loc[condition, :]
    elif axis == 0:
        return df.loc[:, condition]
    else:
        raise ValueError("axis must be 0 or 1")

In [ ]:
path_inputs = "../run-prebuilt-sector-coupled-euro-calliope/build/eurospores/outputs/2016_res_6h_w_noveltech.nc"
model = calliope.read_netcdf(path_inputs)

In [ ]:
def prepare_overview(model, coefficient, techs, aggregate=False):
    df = get_tidy_data(model, coefficient)
    df = filter_techs(df, techs)
    if aggregate:
        df = aggregate_locs(df)
    df = df.set_index("techs", append=True).unstack("techs")
    return df

# df = get_tidy_data(model, "energy_cap")
# df = filter_techs(df, RENEWABLES)
# df = aggregate_locs(df).round(2)
# df.set_index("techs", append=True).unstack("techs")

# df = prepare_overview(model, "energy_cap", RENEWABLES, aggregate=True).round(2)
df = prepare_overview(model, "energy_cap", RENEWABLES, aggregate=True).round(2)
df = df_drop_nan_inf(df, axis=0, drop_if_any=False)
df *= CONVERT_100_GW_to_GW
df

energy_cap                                               \
techs awe_deep_fw1 awe_onshore_sw awe_shallow_fw1 open_field_pv   
locs                                                              
ALB            0.0            0.0             0.0          10.0   
AUT            NaN           15.0             NaN          71.0   
BEL            0.0           10.0             0.0           1.0   
BGR            0.0            0.0             0.0          83.0   
BIH            0.0            0.0             0.0           5.0   
CHE            NaN            0.0             NaN          67.0   
CYP            0.0            0.0             0.0          13.0   
CZE            NaN           50.0             NaN           0.0   
DEU            0.0          153.0             0.0         416.0   
DNK            0.0           27.0             0.0           3.0   
ESP            0.0           43.0             0.0         315.0   
EST            0.0           23.0             0.0          31.0   
FIN            0.0           77.0             0.0           0.0   
FRA            0.0          344.0             0.0         308.0   
GBR            0.0          150.0             0.0         132.0   
GRC            0.0            0.0             0.0          56.0   
HRV            0.0            0.0             0.0          42.0   
HUN            NaN           69.0             NaN           0.0   
IRL            0.0           57.0             0.0           0.0   
ISL            NaN            0.0             NaN           0.0   
ITA            0.0           11.0             0.0         404.0   
LTU            0.0           47.0             0.0           0.0   
LUX            NaN            1.0             NaN           0.0   
LVA            0.0           37.0             0.0           0.0   
MKD            NaN            0.0             NaN           7.0   
MNE            0.0            0.0             0.0           6.0   
NLD            0.0           13.0             0.0          54.0   
NOR            0.0           63.0             0.0           0.0   
POL            0.0          225.0             0.0          37.0   
PRT            0.0           27.0             0.0          64.0   
ROU            0.0           83.0             0.0           0.0   
SRB            NaN            0.0             NaN           0.0   
SVK            NaN            0.0             NaN          19.0   
SVN            0.0            0.0             0.0           0.0   
SWE            0.0          115.0             0.0           0.0   

                                                            \
techs roof_mounted_pv wind_offshore wind_onshore_competing   
locs                                                         
ALB               0.0           0.0                    0.0   
AUT               0.0           0.0                    0.0   
BEL               0.0           0.0                    0.0   
BGR               0.0           0.0                    0.0   
BIH               0.0           0.0                    0.0   
CHE               0.0           0.0                    0.0   
CYP               0.0           0.0                    1.0   
CZE               0.0           0.0                    0.0   
DEU               0.0           0.0                    0.0   
DNK               0.0           0.0                   10.0   
ESP               0.0           0.0                    0.0   
EST               0.0           0.0                    0.0   
FIN               0.0           0.0                    0.0   
FRA               0.0           0.0                    0.0   
GBR               0.0           0.0                    0.0   
GRC               0.0           0.0                    0.0   
HRV               0.0           0.0                    0.0   
HUN               0.0           0.0                    0.0   
IRL               0.0           0.0                    0.0   
ISL               0.0           0.0                    2.0   
ITA               0.0           0.0

In [ ]:
# [x] wind_onshore_monopoly, rooftop_pv: energy_cap_max
#       [ ] power_densities
# [x] open_field_pv, wind_onshore_competing, awe_onshore: available_area
#      [ ] power_densities
# [ ] awe_shallow_fw1, wind_offshore: energy_cap_max group constraint
#      [x] power densities: 8MW/km2
# [ ] awe_deep_fw1, wind_floating: energy_cap_max group constraint
#      [x] power densities: 8MW/km2

In [ ]:
# [x] wind_onshore_monopoly, rooftop_pv: energy_cap_max
def get_energy_cap(model):
    df = prepare_overview(model, "energy_cap_max", RENEWABLES, aggregate=True).round(2)
    df = df_drop_nan_inf(df, axis=0, drop_if_any=False).drop("wind_offshore", axis=1, level=1)
    df *= CONVERT_100_GW_to_GW

    return df

df_energy_cap_max = get_energy_cap(model)
df_energy_cap_max


energy_cap_max                      
techs roof_mounted_pv wind_onshore_monopoly
locs                                       
ALB              14.0                  69.0
AUT              80.0                 137.0
BEL              93.0                  38.0
BGR              76.0                 250.0
BIH              37.0                 151.0
CHE              60.0                  39.0
CYP              20.0                   9.0
CZE              96.0                 204.0
DEU             746.0                 686.0
DNK              67.0                  41.0
ESP             241.0                1026.0
EST              15.0                 180.0
FIN              54.0                1717.0
FRA             746.0                 735.0
GBR             368.0                 546.0
GRC              61.0                 262.0
HRV              65.0                 130.0
HUN              98.0                  89.0
IRL              36.0                 112.0
ISL               0.0                  51.0
ITA             410.0                 492.0
LTU              35.0                 168.0
LUX               5.0                   7.0
LVA              18.0                 251.0
MKD              11.0                  73.0
MNE               7.0                  39.0
NLD             155.0                  25.0
NOR              29.0                1133.0
POL             258.0                 732.0
PRT             118.0                 142.0
ROU             179.0                 461.0
SRB              79.0                 144.0
SVK              54.0                 134.0
SVN              22.0                  51.0
SWE              76.0                2375.0

In [ ]:
#       [ ] power_densities

In [ ]:
# [x] open_field_pv, wind_onshore_competing, awe_onshore: available_area
def get_available_area_onshore(model):
    df = get_tidy_data(model, "available_area").round(2).set_index("locs")
    df = df_drop_nan_inf(df, axis=0, drop_if_any=False) 
    df *= CONVERT_10000km2_to_km2

    return df

df_available_area_onshore = get_available_area_onshore(model)
df_available_area_onshore

,available_area
locs,
LTU_1,23400.0
FRA_12,16700.0
HRV_1,15800.0
LUX_1,700.0
SWE_3,22500.0
...,...
FIN_1,45300.0
ESP_6,6800.0
GBR_6,5200.0


In [ ]:
#      [ ] power_densities

In [ ]:
# [ ] awe_shallow_fw1, wind_offshore: energy_cap_max group constraint
def get_offshore(model):
    selector = list(model.inputs.group_names_energy_cap_max.values)
    model.inputs.sel(group_names_energy_cap_max=selector)[["group_energy_cap_max"]].to_dataframe().unstack(1)
    df = get_tidy_data(model, "group_energy_cap_max").round(2)
    df["group"] = df["group_names_energy_cap_max"].str.extract(r"(wind_offshore_\w+_cap_max)")
    df["locs"] = df["group_names_energy_cap_max"].str.extract(r"wind_offshore_\w+_cap_max_(.*)")
    df = df.drop(columns=["group_names_energy_cap_max"])
    df = df.set_index(["locs", "group"]).unstack("group")

    return df

df_offshore = get_offshore(model)
df_offshore

In [ ]:
joined = df_energy_cap_max.join(df_available_area_onshore).join(df_offshore)
joined

/home/jlauner/.conda/envs/euro-calliope/lib/python3.8/site-packages/pandas/core/frame.py:9130: FutureWarning:

merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)

/home/jlauner/.conda/envs/euro-calliope/lib/python3.8/site-packages/pandas/core/frame.py:9130: FutureWarning:

merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)



,"(energy_cap_max, roof_mounted_pv)","(energy_cap_max, wind_onshore_monopoly)",available_area,"(group_energy_cap_max, wind_offshore_deep_cap_max)","(group_energy_cap_max, wind_offshore_shallow_cap_max)"
locs,,,,,
ALB,14.0,69.0,NaN,NaN,NaN
AUT,80.0,137.0,NaN,NaN,NaN
BEL,93.0,38.0,NaN,NaN,NaN
BGR,76.0,250.0,NaN,NaN,NaN
BIH,37.0,151.0,NaN,NaN,NaN
CHE,60.0,39.0,NaN,NaN,NaN
CYP,20.0,9.0,NaN,NaN,NaN
CZE,96.0,204.0,NaN,NaN,NaN
DEU,746.0,686.0,NaN,NaN,NaN


In [ ]:
prit